In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import numpy as np
import pandas as pd
import re
import nltk
import matplotlib.pyplot as plt

In [3]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report

In [4]:
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.corpus import sentiwordnet as swn
from nltk import ngrams, FreqDist
from nltk.corpus import wordnet

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [5]:
data_neg = pd.read_csv('/content/drive/MyDrive/COL772_A2/training_negative.csv', encoding='ISO-8859-1')
data_pos = pd.read_csv('/content/drive/MyDrive/COL772_A2/training_positive.csv', encoding='ISO-8859-1')

In [6]:
data = data_neg.append(data_pos)
data = data.iloc[:,1:]

In [7]:
data.tail()

,Polarity,Tweet
799995,4,Just woke up. Having no school is the best fee...
799996,4,TheWDB.com - Very cool to hear old Walt interv...
799997,4,Are you ready for your MoJo Makeover? Ask me f...
799998,4,Happy 38th Birthday to my boo of alll time!!! ...
799999,4,happy #charitytuesday @theNSPCC @SparksCharity...


## Sampling Data for Trying out Approaches

In [8]:
df = data.sample(frac=1, random_state=1)
# df = data.iloc[:30000]
df.head()

,Polarity,Tweet
514293,0,i miss nikki nu nu already shes always there ...
142282,0,So I had a dream last night. I remember a sig...
403727,0,@girlyghost ohh poor sickly you (((hugs)) ho...
649503,0,it is raining again
610789,0,@MissKeriBaby wish I was in LA right now


In [9]:
df['Polarity'] = np.where(df['Polarity'] == 4, 1, 0)
df.head()

,Polarity,Tweet
514293,0,i miss nikki nu nu already shes always there ...
142282,0,So I had a dream last night. I remember a sig...
403727,0,@girlyghost ohh poor sickly you (((hugs)) ho...
649503,0,it is raining again
610789,0,@MissKeriBaby wish I was in LA right now


In [10]:
df['Polarity'].value_counts()

1    800000
0    800000
Name: Polarity, dtype: int64

In [11]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

/usr/local/lib/python3.7/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [12]:
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [13]:
example = 'The movie was awesome.'
sid = SentimentIntensityAnalyzer()
sid.polarity_scores(example)

{'compound': 0.6249, 'neg': 0.0, 'neu': 0.423, 'pos': 0.577}

In [14]:
# df[['neg', 'neu', 'pos', 'compound']] = df['Tweet'].apply(sid.polarity_scores).apply(pd.Series)
# df.head()

In [15]:
# df['Polarity_Vader'] = np.where(df['compound'] > 0, 1, 0)
# df.head()

In [16]:
# (sum(df['Polarity'] == df['Polarity_Vader'])/len(df))*100

In [17]:
import seaborn as sns

In [18]:
# for var in ['pos', 'neg', 'neu', 'compound']:
#     plt.figure(figsize=(12,4))
#     sns.distplot(df.query("Polarity==1")[var], bins=30, kde=False, 
#                  color='green', label='Positive')
#     sns.distplot(df.query("Polarity==0")[var], bins=30, kde=False, 
#                  color='red', label='Negative')
#     plt.legend()
#     plt.title(f'Histogram of {var} by true sentiment');

## Pre-Processing Text

In [19]:
def clean_text(tweet):
    tweet = tweet.lower()                                   # Converting to lower case
    tweet = re.sub(r'\b\w+@[^\s]+', ' <MAIL> ', tweet)             # Removing email IDs
    tweet = re.sub(r'@[^\s]+', ' <MENTION> ', tweet)                  # Removing mentions
    tweet = re.sub(r'https?:\/[^\s]+', ' <URL> ', tweet)          # Removing URLs
    tweet = re.sub(r'www.[^\s]+', ' <WEBSITE> ', tweet)               # Removing Websites
    tweet = re.sub(r'\b\w+.com', ' <WEBSITE> ', tweet)             # Removing email IDs
    tweet = re.sub(r'#', ' <HASHTAG> ', tweet)                         # Removing hashtags
    tweet = re.sub(r'_', ' ', tweet)                        # Sometimes hashtags are done with _ representing break between two words
    tweet = re.sub(r'\.{2,}', ' ', tweet)                   # Removing sentence separators
    tweet = re.sub(r"[0-9]+",' ', tweet)                    # Removing numbers as they do not indicate sentiment
    tweet = re.sub(r"\bamp\b", ' ', tweet)                  # Removing &amp signs mis-translated
    tweet = re.sub(r"\bquot\b", ' ', tweet)  
    tweet = re.sub(r"\b\w+;[^\s]+\b", ' ', tweet)  
    if len(tweet) == 0:
      tweet = 'None'
    return ' '.join(tweet.split())

In [20]:
def remove_punc(tweet):
    tweet = re.sub(r"[^\w'\s]+",'', tweet)                  # Removing punctuations apart from clitic
    return tweet

In [21]:
clean_text("I am &amp rachit1jain@gmail.com n't #doing_exceptionally good hello.com &gt;&gt:D") 

"i am & <MAIL> n't <HASHTAG> doing exceptionally good <WEBSITE> &"

In [22]:
clean_text('@')

'@'

In [23]:
def tweet_word_tokenizer(tweet):
    # return word_tokenize(tweet)
    return tweet.split(' ')

In [24]:
clitics = {
    "nt": 'not',
    "ve": 'have',
    "s": 'is',
    "m": 'am',
    "re": 'are',
    "ll": 'will',
    'd': 'would',
    "bout": 'about',
    'didnt': 'did not',
    'havent': 'have not',
    'hasnt': 'has not',
    'wont': 'will not',
    'wouldnt': 'will not',
    'shouldnt': 'should not',
}

In [25]:
# # count = 0
# def handle_clitics(tweet):
#     # global count
#     # count += 1
#     for i in range(len(tweet)):
#         if tweet[i] in clitics.keys():
#             tweet[i] = clitics[tweet[i]]
#     return tweet

In [26]:
def handle_clitics(phrase):
    # specific
    phrase = re.sub(r"won\'t", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)
    phrase = re.sub(r"\bdon't\b", "do not", phrase)
    phrase = re.sub(r"\bdoesn't\b", "does not", phrase)
    phrase = re.sub(r"\bdidn't\b", "did not", phrase)
    phrase = re.sub(r"\bdidnt\b", "did not", phrase)
    phrase = re.sub(r"\bhasn't\b", "has not", phrase)
    phrase = re.sub(r"\bhaven't\b", "have not", phrase)
    phrase = re.sub(r"\bhavent\b", "have not", phrase)
    phrase = re.sub(r"\bhadn't\b", "had not", phrase)
    phrase = re.sub(r"\bwon't\b", "will not", phrase)
    phrase = re.sub(r"\bwouldn't\b", "would not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)

    # using regular expressions to expand the contractions
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)

    return phrase

In [27]:
stop = 0
def stopword_removal(tweet):
    # stoplist = stopwords.words('english')
    stoplist = []
    manual_stoplist = ['retweet', 'retwet', 'rt', 'oh', 'dm', 'mt', 'ht', 'ff', 'shoulda', 'woulda', 'coulda', 'might', 'im', 'tb', 'mysql', 'hah', "a", "an", "the", "and", "but", "if",
                  "or", "because", "as", "until", "while", "of", "at", "by", "for", "with", "about", "against", "between", "into", "through", "during", "before", "after", "above", "below", "to", "from", "up", "down", "in", "out", "on", "off", "over",
                  "under", "again", "further", "then", "once", "here", "there", "when", "where", "why", "how", "all", "any", "both", "each", "few", "more", "most", "other", "some", "such", "nor", "only", "own", "same", "so", "than", "too", "very", "s",
                  "t", "just", "don", "now", 'tweet', 'x', 'f', 'go', 'get', 'give']
    stoplist.append(manual_stoplist)
    global stop
    stop += 1
    if stop % 100 == 0:
      print(stop)
    # stopwords = stopwords.words('english')
    tweet = [word for word in tweet if word not in stoplist]
    if len(tweet) == 0:
        tweet = ['None']
    return tweet

In [28]:
# stopwords.words('english')

In [29]:
short_forms = {
    'n': 'and',
    'ya': 'you',
    'luv': 'love',
    'lol': 'laugh',
    'k': 'okay',
    'na': 'no',
    'ily': 'love',
    'im': 'am',
    'morn': 'morning',
    'nght': 'night',
    'no': 'not',
    'Ill': 'will',
    'shoulda': 'should have'
    }

In [30]:
def handle_shortforms(tweet):
    temp = ''
    for word in tweet.split():
        if word in short_forms.keys():
            temp = temp + ' ' + short_forms[word]
        else:
            temp = temp + ' ' + word
    return ' '.join(temp.split())

In [31]:
# handle_shortforms(['I','am','lol','in','practice'])

In [32]:
handle_shortforms('I am a good boy shoulda gone')

'I am a good boy should have gone'

In [33]:
## Maintaining only letters within a tweet and removing every other information since not indicative of sentiment
def maintain_letters(tweet):
    tweet = re.sub(r'[^a-zA-Z]', ' ', tweet)      # since lowering has already been done
    return ' '.join(tweet.split())

In [34]:
maintain_letters('i am a good boy. hero is @terohja 909')

'i am a good boy hero is terohja'

In [35]:
### TO BE MODIFIED ####
# Emoticons store a lot of information
emo_info = {
    # positive emoticons
    ":‑)": " <EMOJI> ",
    ":)": " <EMOJI> ",
    ";)": " <EMOJI> ",
    ":-}": " <EMOJI> ",
    "=]": " <EMOJI> ",
    "=)": " <EMOJI> ",
    ";d": " <EMOJI> ",
    ":d": " <EMOJI> ",
    ":dd": " <EMOJI> ",
    "xd": " <EMOJI> ",
    "<3": " <EMOJI> ",

    # negativve emoticons
    ":‑(": " <EMOJI> ",
    ":‑[": " <EMOJI> ",
    ":(": " <EMOJI> ",
    "=(": " <EMOJI> ",
    "=/": " <EMOJI> ",
    ":{": " <EMOJI> ",
    ":/": " <EMOJI> ",
    ":|": " <EMOJI> ",
    ":-/": " <EMOJI> ",
    ":o": " <EMOJI> "

}

In [36]:
# def remove_emoji(tweet):
#     ":p": " tease ",
#     "xp": " tease "

In [37]:
### TO BE MODIFIED ####
emo_info_order = [k for (k_len, k) in reversed(sorted([(len(k), k) for k in emo_info.keys()]))]

In [38]:
### TO BE MODIFIED ####
def emo_repl(phrase):
    for k in emo_info_order:
        phrase = phrase.replace(k, emo_info[k])
    return phrase

In [39]:
df['Tweet_regex'] = df['Tweet'].apply(clean_text)
df.head()

,Polarity,Tweet,Tweet_regex
514293,0,i miss nikki nu nu already shes always there ...,i miss nikki nu nu already shes always there w...
142282,0,So I had a dream last night. I remember a sig...,so i had a dream last night. i remember a sign...
403727,0,@girlyghost ohh poor sickly you (((hugs)) ho...,<MENTION> ohh poor sickly you (((hugs)) hope y...
649503,0,it is raining again,it is raining again
610789,0,@MissKeriBaby wish I was in LA right now,<MENTION> wish i was in la right now


In [40]:
# EDA
df[df['Polarity'] == 0]['Tweet_regex'].str.contains('<MENTION>').value_counts()

False    499237
True     300763
Name: Tweet_regex, dtype: int64

In [41]:
df[df['Polarity'] == 4]['Tweet_regex'].str.contains('<MENTION>').value_counts()

Series([], Name: Tweet_regex, dtype: int64)

In [42]:
# EDA
df[df['Polarity'] == 0]['Tweet_regex'].str.contains('<WEBSITE>').value_counts()

False    759160
True      40840
Name: Tweet_regex, dtype: int64

In [43]:
# EDA
df[df['Polarity'] == 4]['Tweet_regex'].str.contains('<WEBSITE>').value_counts()

Series([], Name: Tweet_regex, dtype: int64)

In [44]:
df['Tweet_emoji'] = df['Tweet_regex'].apply(emo_repl)
df.head()

,Polarity,Tweet,Tweet_regex,Tweet_emoji
514293,0,i miss nikki nu nu already shes always there ...,i miss nikki nu nu already shes always there w...,i miss nikki nu nu already shes always there w...
142282,0,So I had a dream last night. I remember a sig...,so i had a dream last night. i remember a sign...,so i had a dream last night. i remember a sign...
403727,0,@girlyghost ohh poor sickly you (((hugs)) ho...,<MENTION> ohh poor sickly you (((hugs)) hope y...,<MENTION> ohh poor sickly you (((hugs)) hope y...
649503,0,it is raining again,it is raining again,it is raining again
610789,0,@MissKeriBaby wish I was in LA right now,<MENTION> wish i was in la right now,<MENTION> wish i was in la right now


In [45]:
df['Tweet_nopunc'] = df['Tweet_emoji'].apply(remove_punc)
# df['Tweet_nopunc'] = df['Tweet_regex'].apply(remove_punc)         # NOT USING EMOJI
df.head()

,Polarity,Tweet,Tweet_regex,Tweet_emoji,Tweet_nopunc
514293,0,i miss nikki nu nu already shes always there ...,i miss nikki nu nu already shes always there w...,i miss nikki nu nu already shes always there w...,i miss nikki nu nu already shes always there w...
142282,0,So I had a dream last night. I remember a sig...,so i had a dream last night. i remember a sign...,so i had a dream last night. i remember a sign...,so i had a dream last night i remember a sign ...
403727,0,@girlyghost ohh poor sickly you (((hugs)) ho...,<MENTION> ohh poor sickly you (((hugs)) hope y...,<MENTION> ohh poor sickly you (((hugs)) hope y...,MENTION ohh poor sickly you hugs hope you feel...
649503,0,it is raining again,it is raining again,it is raining again,it is raining again
610789,0,@MissKeriBaby wish I was in LA right now,<MENTION> wish i was in la right now,<MENTION> wish i was in la right now,MENTION wish i was in la right now


In [46]:
df['Tweet_clitics'] = df['Tweet_nopunc'].apply(handle_clitics)
df.head()

,Polarity,Tweet,Tweet_regex,Tweet_emoji,Tweet_nopunc,Tweet_clitics
514293,0,i miss nikki nu nu already shes always there ...,i miss nikki nu nu already shes always there w...,i miss nikki nu nu already shes always there w...,i miss nikki nu nu already shes always there w...,i miss nikki nu nu already shes always there w...
142282,0,So I had a dream last night. I remember a sig...,so i had a dream last night. i remember a sign...,so i had a dream last night. i remember a sign...,so i had a dream last night i remember a sign ...,so i had a dream last night i remember a sign ...
403727,0,@girlyghost ohh poor sickly you (((hugs)) ho...,<MENTION> ohh poor sickly you (((hugs)) hope y...,<MENTION> ohh poor sickly you (((hugs)) hope y...,MENTION ohh poor sickly you hugs hope you feel...,MENTION ohh poor sickly you hugs hope you feel...
649503,0,it is raining again,it is raining again,it is raining again,it is raining again,it is raining again
610789,0,@MissKeriBaby wish I was in LA right now,<MENTION> wish i was in la right now,<MENTION> wish i was in la right now,MENTION wish i was in la right now,MENTION wish i was in la right now


In [47]:
df['Tweet_shortforms'] = df['Tweet_clitics'].apply(handle_shortforms)
df.head()

,Polarity,Tweet,Tweet_regex,Tweet_emoji,Tweet_nopunc,Tweet_clitics,Tweet_shortforms
514293,0,i miss nikki nu nu already shes always there ...,i miss nikki nu nu already shes always there w...,i miss nikki nu nu already shes always there w...,i miss nikki nu nu already shes always there w...,i miss nikki nu nu already shes always there w...,i miss nikki nu nu already shes always there w...
142282,0,So I had a dream last night. I remember a sig...,so i had a dream last night. i remember a sign...,so i had a dream last night. i remember a sign...,so i had a dream last night i remember a sign ...,so i had a dream last night i remember a sign ...,so i had a dream last night i remember a sign ...
403727,0,@girlyghost ohh poor sickly you (((hugs)) ho...,<MENTION> ohh poor sickly you (((hugs)) hope y...,<MENTION> ohh poor sickly you (((hugs)) hope y...,MENTION ohh poor sickly you hugs hope you feel...,MENTION ohh poor sickly you hugs hope you feel...,MENTION ohh poor sickly you hugs hope you feel...
649503,0,it is raining again,it is raining again,it is raining again,it is raining again,it is raining again,it is raining again
610789,0,@MissKeriBaby wish I was in LA right now,<MENTION> wish i was in la right now,<MENTION> wish i was in la right now,MENTION wish i was in la right now,MENTION wish i was in la right now,MENTION wish i was in la right now


In [48]:
df['Tweet_pure_string'] = df['Tweet_shortforms'].apply(maintain_letters)
df.head()

,Polarity,Tweet,Tweet_regex,Tweet_emoji,Tweet_nopunc,Tweet_clitics,Tweet_shortforms,Tweet_pure_string
514293,0,i miss nikki nu nu already shes always there ...,i miss nikki nu nu already shes always there w...,i miss nikki nu nu already shes always there w...,i miss nikki nu nu already shes always there w...,i miss nikki nu nu already shes always there w...,i miss nikki nu nu already shes always there w...,i miss nikki nu nu already shes always there w...
142282,0,So I had a dream last night. I remember a sig...,so i had a dream last night. i remember a sign...,so i had a dream last night. i remember a sign...,so i had a dream last night i remember a sign ...,so i had a dream last night i remember a sign ...,so i had a dream last night i remember a sign ...,so i had a dream last night i remember a sign ...
403727,0,@girlyghost ohh poor sickly you (((hugs)) ho...,<MENTION> ohh poor sickly you (((hugs)) hope y...,<MENTION> ohh poor sickly you (((hugs)) hope y...,MENTION ohh poor sickly you hugs hope you feel...,MENTION ohh poor sickly you hugs hope you feel...,MENTION ohh poor sickly you hugs hope you feel...,MENTION ohh poor sickly you hugs hope you feel...
649503,0,it is raining again,it is raining again,it is raining again,it is raining again,it is raining again,it is raining again,it is raining again
610789,0,@MissKeriBaby wish I was in LA right now,<MENTION> wish i was in la right now,<MENTION> wish i was in la right now,MENTION wish i was in la right now,MENTION wish i was in la right now,MENTION wish i was in la right now,MENTION wish i was in la right now


In [49]:
df['Tweet_token'] = df['Tweet_pure_string'].apply(tweet_word_tokenizer)
df.head()

,Polarity,Tweet,Tweet_regex,Tweet_emoji,Tweet_nopunc,Tweet_clitics,Tweet_shortforms,Tweet_pure_string,Tweet_token
514293,0,i miss nikki nu nu already shes always there ...,i miss nikki nu nu already shes always there w...,i miss nikki nu nu already shes always there w...,i miss nikki nu nu already shes always there w...,i miss nikki nu nu already shes always there w...,i miss nikki nu nu already shes always there w...,i miss nikki nu nu already shes always there w...,"[i, miss, nikki, nu, nu, already, shes, always..."
142282,0,So I had a dream last night. I remember a sig...,so i had a dream last night. i remember a sign...,so i had a dream last night. i remember a sign...,so i had a dream last night i remember a sign ...,so i had a dream last night i remember a sign ...,so i had a dream last night i remember a sign ...,so i had a dream last night i remember a sign ...,"[so, i, had, a, dream, last, night, i, remembe..."
403727,0,@girlyghost ohh poor sickly you (((hugs)) ho...,<MENTION> ohh poor sickly you (((hugs)) hope y...,<MENTION> ohh poor sickly you (((hugs)) hope y...,MENTION ohh poor sickly you hugs hope you feel...,MENTION ohh poor sickly you hugs hope you feel...,MENTION ohh poor sickly you hugs hope you feel...,MENTION ohh poor sickly you hugs hope you feel...,"[MENTION, ohh, poor, sickly, you, hugs, hope, ..."
649503,0,it is raining again,it is raining again,it is raining again,it is raining again,it is raining again,it is raining again,it is raining again,"[it, is, raining, again]"
610789,0,@MissKeriBaby wish I was in LA right now,<MENTION> wish i was in la right now,<MENTION> wish i was in la right now,MENTION wish i was in la right now,MENTION wish i was in la right now,MENTION wish i was in la right now,MENTION wish i was in la right now,"[MENTION, wish, i, was, in, la, right, now]"


In [64]:
# Was absolutely useless to use
stem = 0
def stemmer(tweet):
    global stem
    stem += 1
    if stem % 1000:
      print(stem)
    porter_stemmer = PorterStemmer()
    tweet = [porter_stemmer.stem(word) for word in tweet]
    return tweet

In [65]:
stemmer(['I','am','playing','making', 'what','I','do'])

1


['I', 'am', 'play', 'make', 'what', 'I', 'do']

In [66]:
df['Tweet_stem'] = df['Tweet_token'].apply(stemmer)
df.head()

Streaming output truncated to the last 5000 lines.
1594996
1594997
1594998
1594999
1595001
1595002
1595003
1595004
1595005
1595006
1595007
1595008
1595009
1595010
1595011
1595012
1595013
1595014
1595015
1595016
1595017
1595018
1595019
1595020
1595021
1595022
1595023
1595024
1595025
1595026
1595027
1595028
1595029
1595030
1595031
1595032
1595033
1595034
1595035
1595036
1595037
1595038
1595039
1595040
1595041
1595042
1595043
1595044
1595045
1595046
1595047
1595048
1595049
1595050
1595051
1595052
1595053
1595054
1595055
1595056
1595057
1595058
1595059
1595060
1595061
1595062
1595063
1595064
1595065
1595066
1595067
1595068
1595069
1595070
1595071
1595072
1595073
1595074
1595075
1595076
1595077
1595078
1595079
1595080
1595081
1595082
1595083
1595084
1595085
1595086
1595087
1595088
1595089
1595090
1595091
1595092
1595093
1595094
1595095
1595096
1595097
1595098
1595099
1595100
1595101
1595102
1595103
1595104
1595105
1595106
1595107
1595108
1595109
1595110
1595111
1595112
1595113
1595114
15951

,Polarity,Tweet,Tweet_regex,Tweet_emoji,Tweet_nopunc,Tweet_clitics,Tweet_shortforms,Tweet_pure_string,Tweet_token,Tweet_stem
514293,0,i miss nikki nu nu already shes always there ...,i miss nikki nu nu already shes always there w...,i miss nikki nu nu already shes always there w...,i miss nikki nu nu already shes always there w...,i miss nikki nu nu already shes always there w...,i miss nikki nu nu already shes always there w...,i miss nikki nu nu already shes always there w...,"[i, miss, nikki, nu, nu, already, shes, always...","[i, miss, nikki, nu, nu, alreadi, she, alway, ..."
142282,0,So I had a dream last night. I remember a sig...,so i had a dream last night. i remember a sign...,so i had a dream last night. i remember a sign...,so i had a dream last night i remember a sign ...,so i had a dream last night i remember a sign ...,so i had a dream last night i remember a sign ...,so i had a dream last night i remember a sign ...,"[so, i, had, a, dream, last, night, i, remembe...","[so, i, had, a, dream, last, night, i, rememb,..."
403727,0,@girlyghost ohh poor sickly you (((hugs)) ho...,<MENTION> ohh poor sickly you (((hugs)) hope y...,<MENTION> ohh poor sickly you (((hugs)) hope y...,MENTION ohh poor sickly you hugs hope you feel...,MENTION ohh poor sickly you hugs hope you feel...,MENTION ohh poor sickly you hugs hope you feel...,MENTION ohh poor sickly you hugs hope you feel...,"[MENTION, ohh, poor, sickly, you, hugs, hope, ...","[mention, ohh, poor, sickli, you, hug, hope, y..."
649503,0,it is raining again,it is raining again,it is raining again,it is raining again,it is raining again,it is raining again,it is raining again,"[it, is, raining, again]","[it, is, rain, again]"
610789,0,@MissKeriBaby wish I was in LA right now,<MENTION> wish i was in la right now,<MENTION> wish i was in la right now,MENTION wish i was in la right now,MENTION wish i was in la right now,MENTION wish i was in la right now,MENTION wish i was in la right now,"[MENTION, wish, i, was, in, la, right, now]","[mention, wish, i, wa, in, la, right, now]"


In [68]:
def make_sentences(df, col, title):
    df[title] = df[col].apply(lambda x:' '.join([i for i in x]))
    return df

In [69]:
df = make_sentences(df, 'Tweet_stem', 'Tweet_sent')
df.head()

,Polarity,Tweet,Tweet_regex,Tweet_emoji,Tweet_nopunc,Tweet_clitics,Tweet_shortforms,Tweet_pure_string,Tweet_token,Tweet_stem,Tweet_sent
514293,0,i miss nikki nu nu already shes always there ...,i miss nikki nu nu already shes always there w...,i miss nikki nu nu already shes always there w...,i miss nikki nu nu already shes always there w...,i miss nikki nu nu already shes always there w...,i miss nikki nu nu already shes always there w...,i miss nikki nu nu already shes always there w...,"[i, miss, nikki, nu, nu, already, shes, always...","[i, miss, nikki, nu, nu, alreadi, she, alway, ...",i miss nikki nu nu alreadi she alway there whe...
142282,0,So I had a dream last night. I remember a sig...,so i had a dream last night. i remember a sign...,so i had a dream last night. i remember a sign...,so i had a dream last night i remember a sign ...,so i had a dream last night i remember a sign ...,so i had a dream last night i remember a sign ...,so i had a dream last night i remember a sign ...,"[so, i, had, a, dream, last, night, i, remembe...","[so, i, had, a, dream, last, night, i, rememb,...",so i had a dream last night i rememb a sign wh...
403727,0,@girlyghost ohh poor sickly you (((hugs)) ho...,<MENTION> ohh poor sickly you (((hugs)) hope y...,<MENTION> ohh poor sickly you (((hugs)) hope y...,MENTION ohh poor sickly you hugs hope you feel...,MENTION ohh poor sickly you hugs hope you feel...,MENTION ohh poor sickly you hugs hope you feel...,MENTION ohh poor sickly you hugs hope you feel...,"[MENTION, ohh, poor, sickly, you, hugs, hope, ...","[mention, ohh, poor, sickli, you, hug, hope, y...",mention ohh poor sickli you hug hope you feel ...
649503,0,it is raining again,it is raining again,it is raining again,it is raining again,it is raining again,it is raining again,it is raining again,"[it, is, raining, again]","[it, is, rain, again]",it is rain again
610789,0,@MissKeriBaby wish I was in LA right now,<MENTION> wish i was in la right now,<MENTION> wish i was in la right now,MENTION wish i was in la right now,MENTION wish i was in la right now,MENTION wish i was in la right now,MENTION wish i was in la right now,"[MENTION, wish, i, was, in, la, right, now]","[mention, wish, i, wa, in, la, right, now]",mention wish i wa in la right now


In [70]:
def pos_tagger(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:         
        return None

In [71]:
#### TO BE MODIFIED ######
count = 0
def pos_tagging(tweet):
    global count
    count += 1
    if count % 100 == 0:
      print(count)
    # tweet = nltk.pos_tag(tweet) 
    tweet = nltk.pos_tag([i for i in tweet if i])
    return tweet

In [72]:
pos_tagging(['','I','am','good'])

[('I', 'PRP'), ('am', 'VBP'), ('good', 'JJ')]

In [73]:
### TO BE MODIFIED ################
def tweet_lemmatizer(tweet):
    lemmatizer = WordNetLemmatizer()
    lemmatized = []
    pos_wordnet = list(map(lambda x: (x[0], pos_tagger(x[1])), tweet))
    for word, tag in pos_wordnet:
        if tag is None:
            lemmatized.append(word)
        else:       
            lemmatized.append(lemmatizer.lemmatize(word, tag))
    lemmatized = ' '.join(lemmatized)
    lemmatized_sent = ', '.join(lemmatized)
    # print(list(lemmatized.split()))
    return list(lemmatized.split())

In [74]:
# df['Tweet_pos'] = df['Tweet_token'].apply(pos_tagging)
# df.head()

In [75]:
# df['Tweet_pos'] = df['Tweet_token'].apply(pos_tagging)
# df.head()

In [76]:
# df['Tweet_lemma'] = df['Tweet_pos'].apply(tweet_lemmatizer)
# df.head()

In [77]:
def make_sentences(df, col, title):
    df[title] = df[col].apply(lambda x:' '.join([i for i in x]))
    return df

In [78]:
# df = make_sentences(df, 'Tweet_lemma', 'Tweet_sent')
# df.head()

In [79]:
wordnet.synsets('hello')

KeyboardInterrupt: ignored

In [80]:
## TO BE MODIFIED #####
def normalisation_words(tweet):
    tweet = tweet.replace(r'([a-z])\1{1,}', r'\1\1')
    # tweet = re.sub(r'(ha){1,}', r'laugh', tweet)
    tweet = re.sub(r'h+a+[ha]+', r'laugh', tweet)
    # tweet = re.sub(r'(lol){1,}', r'laugh', tweet)
    tweet = re.sub(r'l+o+[lo]+', r'laugh', tweet)
    tweet = ' '.join([word if len(wordnet.synsets(word)) > 0 else re.sub(r'([a-z])\1{1,}', r'\1\1', word) for word in tweet.split()])
    tweet = re.sub(r'\b([a-z])\1{1,}', r' ', tweet)     # If only repeated letters are left, remove them
    tweet = re.sub(r"\b[a-zA-Z]{1}\b", ' ', tweet)        # Removing single letters
    # tweet = re.sub(r'lo+l+o+[^\s]+', r'lol', tweet)
    return tweet.split()

In [82]:
normalisation_words('awww hahahhhahahahahahahhahaaaaaaaaaaahhhahaha ppeeee lollipop looooooool happppiest day lolllll lll lool bummer get david carr third day laugh')

['aww',
 'laugh',
 'ee',
 'laughipop',
 'laugh',
 'happiest',
 'day',
 'laugh',
 'laugh',
 'bummer',
 'get',
 'david',
 'carr',
 'third',
 'day',
 'laugh']

In [83]:
df['Tweet_normalised'] = df['Tweet_sent'].apply(normalisation_words)
df.head()

,Polarity,Tweet,Tweet_regex,Tweet_emoji,Tweet_nopunc,Tweet_clitics,Tweet_shortforms,Tweet_pure_string,Tweet_token,Tweet_stem,Tweet_sent,Tweet_normalised
514293,0,i miss nikki nu nu already shes always there ...,i miss nikki nu nu already shes always there w...,i miss nikki nu nu already shes always there w...,i miss nikki nu nu already shes always there w...,i miss nikki nu nu already shes always there w...,i miss nikki nu nu already shes always there w...,i miss nikki nu nu already shes always there w...,"[i, miss, nikki, nu, nu, already, shes, always...","[i, miss, nikki, nu, nu, alreadi, she, alway, ...",i miss nikki nu nu alreadi she alway there whe...,"[miss, nikki, nu, nu, alreadi, she, alway, the..."
142282,0,So I had a dream last night. I remember a sig...,so i had a dream last night. i remember a sign...,so i had a dream last night. i remember a sign...,so i had a dream last night i remember a sign ...,so i had a dream last night i remember a sign ...,so i had a dream last night i remember a sign ...,so i had a dream last night i remember a sign ...,"[so, i, had, a, dream, last, night, i, remembe...","[so, i, had, a, dream, last, night, i, rememb,...",so i had a dream last night i rememb a sign wh...,"[so, had, dream, last, night, rememb, sign, wh..."
403727,0,@girlyghost ohh poor sickly you (((hugs)) ho...,<MENTION> ohh poor sickly you (((hugs)) hope y...,<MENTION> ohh poor sickly you (((hugs)) hope y...,MENTION ohh poor sickly you hugs hope you feel...,MENTION ohh poor sickly you hugs hope you feel...,MENTION ohh poor sickly you hugs hope you feel...,MENTION ohh poor sickly you hugs hope you feel...,"[MENTION, ohh, poor, sickly, you, hugs, hope, ...","[mention, ohh, poor, sickli, you, hug, hope, y...",mention ohh poor sickli you hug hope you feel ...,"[mention, ohh, poor, sickli, you, hug, hope, y..."
649503,0,it is raining again,it is raining again,it is raining again,it is raining again,it is raining again,it is raining again,it is raining again,"[it, is, raining, again]","[it, is, rain, again]",it is rain again,"[it, is, rain, again]"
610789,0,@MissKeriBaby wish I was in LA right now,<MENTION> wish i was in la right now,<MENTION> wish i was in la right now,MENTION wish i was in la right now,MENTION wish i was in la right now,MENTION wish i was in la right now,MENTION wish i was in la right now,"[MENTION, wish, i, was, in, la, right, now]","[mention, wish, i, wa, in, la, right, now]",mention wish i wa in la right now,"[mention, wish, wa, in, la, right, now]"


In [84]:
df['Tweet_stopword'] = df['Tweet_normalised'].apply(stopword_removal)
df.head()

Streaming output truncated to the last 5000 lines.
1100100
1100200
1100300
1100400
1100500
1100600
1100700
1100800
1100900
1101000
1101100
1101200
1101300
1101400
1101500
1101600
1101700
1101800
1101900
1102000
1102100
1102200
1102300
1102400
1102500
1102600
1102700
1102800
1102900
1103000
1103100
1103200
1103300
1103400
1103500
1103600
1103700
1103800
1103900
1104000
1104100
1104200
1104300
1104400
1104500
1104600
1104700
1104800
1104900
1105000
1105100
1105200
1105300
1105400
1105500
1105600
1105700
1105800
1105900
1106000
1106100
1106200
1106300
1106400
1106500
1106600
1106700
1106800
1106900
1107000
1107100
1107200
1107300
1107400
1107500
1107600
1107700
1107800
1107900
1108000
1108100
1108200
1108300
1108400
1108500
1108600
1108700
1108800
1108900
1109000
1109100
1109200
1109300
1109400
1109500
1109600
1109700
1109800
1109900
1110000
1110100
1110200
1110300
1110400
1110500
1110600
1110700
1110800
1110900
1111000
1111100
1111200
1111300
1111400
1111500
1111600
1111700
1111800
11119

,Polarity,Tweet,Tweet_regex,Tweet_emoji,Tweet_nopunc,Tweet_clitics,Tweet_shortforms,Tweet_pure_string,Tweet_token,Tweet_stem,Tweet_sent,Tweet_normalised,Tweet_stopword
514293,0,i miss nikki nu nu already shes always there ...,i miss nikki nu nu already shes always there w...,i miss nikki nu nu already shes always there w...,i miss nikki nu nu already shes always there w...,i miss nikki nu nu already shes always there w...,i miss nikki nu nu already shes always there w...,i miss nikki nu nu already shes always there w...,"[i, miss, nikki, nu, nu, already, shes, always...","[i, miss, nikki, nu, nu, alreadi, she, alway, ...",i miss nikki nu nu alreadi she alway there whe...,"[miss, nikki, nu, nu, alreadi, she, alway, the...","[miss, nikki, nu, nu, alreadi, she, alway, the..."
142282,0,So I had a dream last night. I remember a sig...,so i had a dream last night. i remember a sign...,so i had a dream last night. i remember a sign...,so i had a dream last night i remember a sign ...,so i had a dream last night i remember a sign ...,so i had a dream last night i remember a sign ...,so i had a dream last night i remember a sign ...,"[so, i, had, a, dream, last, night, i, remembe...","[so, i, had, a, dream, last, night, i, rememb,...",so i had a dream last night i rememb a sign wh...,"[so, had, dream, last, night, rememb, sign, wh...","[so, had, dream, last, night, rememb, sign, wh..."
403727,0,@girlyghost ohh poor sickly you (((hugs)) ho...,<MENTION> ohh poor sickly you (((hugs)) hope y...,<MENTION> ohh poor sickly you (((hugs)) hope y...,MENTION ohh poor sickly you hugs hope you feel...,MENTION ohh poor sickly you hugs hope you feel...,MENTION ohh poor sickly you hugs hope you feel...,MENTION ohh poor sickly you hugs hope you feel...,"[MENTION, ohh, poor, sickly, you, hugs, hope, ...","[mention, ohh, poor, sickli, you, hug, hope, y...",mention ohh poor sickli you hug hope you feel ...,"[mention, ohh, poor, sickli, you, hug, hope, y...","[mention, ohh, poor, sickli, you, hug, hope, y..."
649503,0,it is raining again,it is raining again,it is raining again,it is raining again,it is raining again,it is raining again,it is raining again,"[it, is, raining, again]","[it, is, rain, again]",it is rain again,"[it, is, rain, again]","[it, is, rain, again]"
610789,0,@MissKeriBaby wish I was in LA right now,<MENTION> wish i was in la right now,<MENTION> wish i was in la right now,MENTION wish i was in la right now,MENTION wish i was in la right now,MENTION wish i was in la right now,MENTION wish i was in la right now,"[MENTION, wish, i, was, in, la, right, now]","[mention, wish, i, wa, in, la, right, now]",mention wish i wa in la right now,"[mention, wish, wa, in, la, right, now]","[mention, wish, wa, in, la, right, now]"


In [86]:
df = make_sentences(df, 'Tweet_stopword', 'Tweet_final_sent')
df.head()

,Polarity,Tweet,Tweet_regex,Tweet_emoji,Tweet_nopunc,Tweet_clitics,Tweet_shortforms,Tweet_pure_string,Tweet_token,Tweet_stem,Tweet_sent,Tweet_normalised,Tweet_stopword,Tweet_final_sent
514293,0,i miss nikki nu nu already shes always there ...,i miss nikki nu nu already shes always there w...,i miss nikki nu nu already shes always there w...,i miss nikki nu nu already shes always there w...,i miss nikki nu nu already shes always there w...,i miss nikki nu nu already shes always there w...,i miss nikki nu nu already shes always there w...,"[i, miss, nikki, nu, nu, already, shes, always...","[i, miss, nikki, nu, nu, alreadi, she, alway, ...",i miss nikki nu nu alreadi she alway there whe...,"[miss, nikki, nu, nu, alreadi, she, alway, the...","[miss, nikki, nu, nu, alreadi, she, alway, the...",miss nikki nu nu alreadi she alway there when ...
142282,0,So I had a dream last night. I remember a sig...,so i had a dream last night. i remember a sign...,so i had a dream last night. i remember a sign...,so i had a dream last night i remember a sign ...,so i had a dream last night i remember a sign ...,so i had a dream last night i remember a sign ...,so i had a dream last night i remember a sign ...,"[so, i, had, a, dream, last, night, i, remembe...","[so, i, had, a, dream, last, night, i, rememb,...",so i had a dream last night i rememb a sign wh...,"[so, had, dream, last, night, rememb, sign, wh...","[so, had, dream, last, night, rememb, sign, wh...",so had dream last night rememb sign which clea...
403727,0,@girlyghost ohh poor sickly you (((hugs)) ho...,<MENTION> ohh poor sickly you (((hugs)) hope y...,<MENTION> ohh poor sickly you (((hugs)) hope y...,MENTION ohh poor sickly you hugs hope you feel...,MENTION ohh poor sickly you hugs hope you feel...,MENTION ohh poor sickly you hugs hope you feel...,MENTION ohh poor sickly you hugs hope you feel...,"[MENTION, ohh, poor, sickly, you, hugs, hope, ...","[mention, ohh, poor, sickli, you, hug, hope, y...",mention ohh poor sickli you hug hope you feel ...,"[mention, ohh, poor, sickli, you, hug, hope, y...","[mention, ohh, poor, sickli, you, hug, hope, y...",mention ohh poor sickli you hug hope you feel ...
649503,0,it is raining again,it is raining again,it is raining again,it is raining again,it is raining again,it is raining again,it is raining again,"[it, is, raining, again]","[it, is, rain, again]",it is rain again,"[it, is, rain, again]","[it, is, rain, again]",it is rain again
610789,0,@MissKeriBaby wish I was in LA right now,<MENTION> wish i was in la right now,<MENTION> wish i was in la right now,MENTION wish i was in la right now,MENTION wish i was in la right now,MENTION wish i was in la right now,MENTION wish i was in la right now,"[MENTION, wish, i, was, in, la, right, now]","[mention, wish, i, wa, in, la, right, now]",mention wish i wa in la right now,"[mention, wish, wa, in, la, right, now]","[mention, wish, wa, in, la, right, now]",mention wish wa in la right now


In [87]:
df.drop(df[df["Tweet_final_sent"] == ''].index, inplace=True)
df = df.reset_index(drop=True)

In [88]:
df.head()

,Polarity,Tweet,Tweet_regex,Tweet_emoji,Tweet_nopunc,Tweet_clitics,Tweet_shortforms,Tweet_pure_string,Tweet_token,Tweet_stem,Tweet_sent,Tweet_normalised,Tweet_stopword,Tweet_final_sent
0,0,i miss nikki nu nu already shes always there ...,i miss nikki nu nu already shes always there w...,i miss nikki nu nu already shes always there w...,i miss nikki nu nu already shes always there w...,i miss nikki nu nu already shes always there w...,i miss nikki nu nu already shes always there w...,i miss nikki nu nu already shes always there w...,"[i, miss, nikki, nu, nu, already, shes, always...","[i, miss, nikki, nu, nu, alreadi, she, alway, ...",i miss nikki nu nu alreadi she alway there whe...,"[miss, nikki, nu, nu, alreadi, she, alway, the...","[miss, nikki, nu, nu, alreadi, she, alway, the...",miss nikki nu nu alreadi she alway there when ...
1,0,So I had a dream last night. I remember a sig...,so i had a dream last night. i remember a sign...,so i had a dream last night. i remember a sign...,so i had a dream last night i remember a sign ...,so i had a dream last night i remember a sign ...,so i had a dream last night i remember a sign ...,so i had a dream last night i remember a sign ...,"[so, i, had, a, dream, last, night, i, remembe...","[so, i, had, a, dream, last, night, i, rememb,...",so i had a dream last night i rememb a sign wh...,"[so, had, dream, last, night, rememb, sign, wh...","[so, had, dream, last, night, rememb, sign, wh...",so had dream last night rememb sign which clea...
2,0,@girlyghost ohh poor sickly you (((hugs)) ho...,<MENTION> ohh poor sickly you (((hugs)) hope y...,<MENTION> ohh poor sickly you (((hugs)) hope y...,MENTION ohh poor sickly you hugs hope you feel...,MENTION ohh poor sickly you hugs hope you feel...,MENTION ohh poor sickly you hugs hope you feel...,MENTION ohh poor sickly you hugs hope you feel...,"[MENTION, ohh, poor, sickly, you, hugs, hope, ...","[mention, ohh, poor, sickli, you, hug, hope, y...",mention ohh poor sickli you hug hope you feel ...,"[mention, ohh, poor, sickli, you, hug, hope, y...","[mention, ohh, poor, sickli, you, hug, hope, y...",mention ohh poor sickli you hug hope you feel ...
3,0,it is raining again,it is raining again,it is raining again,it is raining again,it is raining again,it is raining again,it is raining again,"[it, is, raining, again]","[it, is, rain, again]",it is rain again,"[it, is, rain, again]","[it, is, rain, again]",it is rain again
4,0,@MissKeriBaby wish I was in LA right now,<MENTION> wish i was in la right now,<MENTION> wish i was in la right now,MENTION wish i was in la right now,MENTION wish i was in la right now,MENTION wish i was in la right now,MENTION wish i was in la right now,"[MENTION, wish, i, was, in, la, right, now]","[mention, wish, i, wa, in, la, right, now]",mention wish i wa in la right now,"[mention, wish, wa, in, la, right, now]","[mention, wish, wa, in, la, right, now]",mention wish wa in la right now


In [90]:
# Think if you want to do stratify
X_train, X_test, y_train, y_test = train_test_split(df['Tweet_final_sent'], df['Polarity'], stratify=df['Polarity'], test_size=0.1, random_state=2)

In [91]:
# X_train, X_dev, y_train, y_dev = train_test_split(X_train, y_train, test_size=0.1, random_state=2)

In [92]:
X_train.shape

(1440000,)

In [93]:
X_test.shape

(160000,)

In [94]:
# X_dev.shape

In [95]:
X_train


218210     mention perez websit ing out of the closet and...
1265560    is feel tire doe not want to go back to work t...
133517     my favorti red head is in the hospit love you ...
1224133    mention you are right will still go check it o...
1302305    mention not worri have got tast make movi almo...
                                 ...                        
1529526                         mention how yu do today dear
580848                                 king of leonmi obsess
120795     well it not sunni up here in leed it now bit b...
240277     just back from my granni all the famili togeth...
1052118    philosophi for me again today buuh do not want...
Name: Tweet_final_sent, Length: 1440000, dtype: object

In [96]:
import csv, collections
from sklearn.base import BaseEstimator
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import FeatureUnion
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
import numpy as np
from sklearn.metrics import roc_curve, auc

In [97]:
def load_sent_word_net():
    sent_scores = collections.defaultdict(list)

    with open("../content/drive/MyDrive/COL772_A2/SentiWordNet_3.0.0.txt","r") as csvfile:
        reader = csv.reader(csvfile, delimiter='\t', quotechar='"')

        for line in reader:
            if line[0].startswith("#"):
                continue
            if len(line) == 1:
                continue
            POS, ID, PosScore, NegScore, SynsetTerms, Glos = line
            if len(POS) == 0 or len(ID) == 0:
                continue
            for term in SynsetTerms.split(" "):
                term = term.split('#')[0]
                # print(term)
                term = term.replace("-", " ").replace("_", " ")
                key = "%s/%s" % (POS, term)
                # print(key)
                sent_scores[key].append((float(PosScore), float(NegScore)))
                # print(sent_scores)
        for key, value in sent_scores.items():
            sent_scores[key] = np.mean(value, axis=0)

        return sent_scores


sent_word_net = load_sent_word_net()

In [98]:
class LinguisticVectorizer(BaseEstimator):

    def get_feature_names(self):
        return np.array(['sent_pos', 'sent_neg', 'nouns', 'adjectives', 'verbs', 'adverbs'])

    def fit(self, documents, y=None):
        return self

    def _get_sentiments(self, d):
        sent = tuple(d.split())
        tagged = nltk.pos_tag(sent)

        pos_vals = []
        neg_vals = []

        nouns = 0.
        adjectives = 0.
        verbs = 0.
        adverbs = 0.

        i = 0
        for w, t in tagged:

            p, n = 0, 0
            sent_pos_type = None
            if t.startswith("NN"):
                #noun
                sent_pos_type = "n"
                nouns += 1
            elif t.startswith("JJ"):
                #adjective
                sent_pos_type = "a"
                adjectives += 1
            elif t.startswith("VB"):
                #verb
                sent_pos_type = "v"
                verbs += 1
            elif t.startswith("RB"):
                #adverb
                sent_pos_type = "r"
                adverbs += 1
            else:
                sent_pos_type = "Nan"

                i += 1
                l = len(sent) - i

                if l == 0:
                    l = 1
                else:
                    pass

            if sent_pos_type is not None:

                sent_word = "%s/%s" % (sent_pos_type, w)

                if sent_word in sent_word_net:
                    p, n = sent_word_net[sent_word]
                elif sent_word == "Nan":
                    p, n = 0, 0

                pos_vals.append(p)
                neg_vals.append(n)

        if i == 0:
            l = len(sent)
        else:
            pass

        avg_pos_val = np.mean(pos_vals)
        avg_neg_val = np.mean(neg_vals)

        return [avg_pos_val, avg_neg_val, nouns / l, adjectives / l, verbs / l, adverbs / l]

    # print(_get_sentiments('This be fantastic'))

    def transform(self, documents):
        pos_val, neg_val, nouns, adjectives, verbs, adverbs = np.array([self._get_sentiments(d) for d in documents]).T
        result = np.array([pos_val, neg_val, nouns, adjectives, verbs, adverbs]).T

        return result

In [99]:
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV, RandomizedSearchCV

In [100]:
# Define functions
def create_baseline_models():
    """Create list of baseline models."""
    models = []
    models.append(('log', LogisticRegression(random_state=123, 
                                             max_iter=1000)))
    models.append(('sgd', SGDClassifier(random_state=123)))
    models.append(('mnb', MultinomialNB()))
    return models

def assess(X, y, models, cv=5, scoring=['roc_auc', 
                                        'accuracy', 
                                        'f1']):
    """Provide summary of cross validation results for models."""
    results = pd.DataFrame()
    for name, model in models:
        result = pd.DataFrame(cross_validate(model, X, y, cv=cv, 
                                             scoring=scoring))
        mean = result.mean().rename('{}_mean'.format)
        std = result.std().rename('{}_std'.format)
        results[name] = pd.concat([mean, std], axis=0)
    return results.sort_index()

In [101]:
models = create_baseline_models()
models

[('log',
  LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                     intercept_scaling=1, l1_ratio=None, max_iter=1000,
                     multi_class='auto', n_jobs=None, penalty='l2',
                     random_state=123, solver='lbfgs', tol=0.0001, verbose=0,
                     warm_start=False)),
 ('sgd', SGDClassifier(alpha=0.0001, average=False, class_weight=None,
                early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
                l1_ratio=0.15, learning_rate='optimal', loss='hinge',
                max_iter=1000, n_iter_no_change=5, n_jobs=None, penalty='l2',
                power_t=0.5, random_state=123, shuffle=True, tol=0.001,
                validation_fraction=0.1, verbose=0, warm_start=False)),
 ('mnb', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))]

In [102]:
# Preprocess the data
vectoriser = TfidfVectorizer(token_pattern=r'[a-z]+', 
                             stop_words='english', 
                             min_df=30, 
                             max_df=.7)
X_train_simpler = vectoriser.fit_transform(X_train)
# Assess the model
assess(X_train_simpler, y_train, models)

,log,sgd,mnb
fit_time_mean,60.548713,3.056078,0.274333
fit_time_std,5.395908,0.017050,0.002879
score_time_mean,0.303290,0.255893,0.306942
score_time_std,0.029923,0.003329,0.004685
test_accuracy_mean,0.770774,0.760117,0.756687
test_accuracy_std,0.000784,0.000636,0.000849
test_f1_mean,0.775745,0.770988,0.758894
test_f1_std,0.000634,0.000738,0.000708
test_roc_auc_mean,0.850479,0.837984,0.837284
test_roc_auc_std,0.000718,0.000448,0.000996


In [103]:
# Create a pipeline
pipe = Pipeline([('vectoriser', TfidfVectorizer(token_pattern=r'[a-z]+')),
                 ('model', SGDClassifier(random_state=123))])
# Prepare a random search
param_distributions = {'vectoriser__min_df': np.arange(10, 1000, 10),
                       'vectoriser__max_df': np.linspace(.2, 1, 40),
                       'model__loss': ['log', 'hinge']}
r_search = RandomizedSearchCV(estimator=pipe, param_distributions=param_distributions, 
                              n_iter=30, cv=5, n_jobs=-1, random_state=123)
r_search.fit(X_train, y_train)
# Save results to a dataframe
r_search_results = pd.DataFrame(r_search.cv_results_).sort_values(by='rank_test_score')

KeyboardInterrupt: ignored

In [ ]:
columns = [col for col in r_search_results.columns 
           if re.search(r"split|param_", col)]
r_summary = r_search_results[columns].copy()
r_summary.columns = [re.sub(r'_test_score|param_', '', col) 
                     for col in r_summary.columns]
columns = [col.split('__')[1] if '__' in col else col 
           for col in r_summary.columns ]
r_summary.columns = columns
r_summary.head()

In [ ]:
# Create a pipeline
pipe = Pipeline([('vectoriser', TfidfVectorizer(token_pattern=r'[a-z]+', max_df=.6)),
                 ('model', SGDClassifier(random_state=123, loss='hinge'))])
# Prepare a grid search
param_grid = {'vectoriser__min_df': [30, 90, 150],
              'vectoriser__ngram_range': [(1,1), (1,2)],
              'vectoriser__stop_words': [None, 'english'],
              'model__fit_intercept': [True, False]}
g_search = GridSearchCV(estimator=pipe, param_grid=param_grid, cv=5, n_jobs=-1)
g_search.fit(X_train, y_train)
# Save results to a dataframe
g_search_results = pd.DataFrame(g_search.cv_results_).sort_values(by='rank_test_score')

In [ ]:
# columns = [col for col in g_search_results.columns 
#            if re.search(r"split|param_", col)]
# g_summary = g_search_results[columns+['mean_test_score']].copy()
# g_summary.columns = [re.sub(r'_test_score|param_', '', col) 
#                      for col in g_summary.columns]
# columns = [col.split('__')[1] if '__' in col else col 
#            for col in g_summary.columns ]
# g_summary.columns = columns
# g_summary.head()

In [ ]:
# # Create a long dataframe
# g_summary_long = pd.melt(g_summary, 
#                          id_vars=['min_df', 
#                                   'ngram_range', 
#                                   'stop_words', 
#                                   'fit_intercept'], 
#                          value_vars=['split0', 
#                                      'split1', 
#                                      'split2', 
#                                      'split3', 
#                                      'split4'])
# g_summary_long.replace({None: 'None'}, inplace=True)
# # Plot performance
# for param in ['ngram_range', 'stop_words', 'fit_intercept']:
#     plt.figure(figsize=(8,4))
#     plt.title(f'Performance by {param}')
#     sns.boxplot(x='value', y=param, data=g_summary_long, orient='h')
#     plt.xlim(.85, .95);

In [104]:
pipe = Pipeline([('vectoriser', TfidfVectorizer(token_pattern=r'[a-z]+', min_df=30, max_df=.6, ngram_range=(1,2))),
                 ('model', SGDClassifier(random_state=123, loss='hinge'))])
pipe.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('vectoriser',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=0.6, max_features=None,
                                 min_df=30, ngram_range=(1, 2), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False, token_pat...
                ('model',
                 SGDClassifier(alpha=0.0001, average=False, class_weight=None,
                               early_stopping=False, epsilon=0.1, eta0=0.0,
                               fit_intercept=True, l1_ratio=0.15,
                               learning_rate='optimal', loss='hinge',
                  

In [105]:
coefs = pd.DataFrame(pipe['model'].coef_, 
                     columns=pipe['vectoriser'].get_feature_names())
coefs = coefs.T.rename(columns={0:'coef'}).sort_values('coef')
coefs

,coef
not,-7.286392
miss,-6.390041
sad,-5.612452
wish,-4.155137
hate,-3.909768
...,...
laugh,2.780690
good,2.985263
love,3.242226
thank,3.300331


In [ ]:
train_pred = pipe.predict(X_train)
print(classification_report(train_pred, 
                            y_train, 
                            target_names=['negative', 'positive']))

In [107]:
test_pred = pipe.predict(X_test)
print(classification_report(test_pred, 
                            y_test, 
                            target_names=['negative', 'positive']))

              precision    recall  f1-score   support

    negative       0.78      0.80      0.79     77463
    positive       0.81      0.78      0.80     82537

    accuracy                           0.79    160000
   macro avg       0.79      0.79      0.79    160000
weighted avg       0.79      0.79      0.79    160000



In [207]:
for i in range(10):
    lead = X_test.sample(1)
    %timeit pipe.predict(lead)

1000 loops, best of 5: 674 µs per loop
1000 loops, best of 5: 680 µs per loop
1000 loops, best of 5: 669 µs per loop
1000 loops, best of 5: 677 µs per loop
1000 loops, best of 5: 688 µs per loop
1000 loops, best of 5: 653 µs per loop
1000 loops, best of 5: 697 µs per loop
1000 loops, best of 5: 672 µs per loop
1000 loops, best of 5: 664 µs per loop
1000 loops, best of 5: 671 µs per loop


In [208]:
# tfidf_ngrams = TfidfVectorizer(min_df=5, ngram_range=(1, 3))
# ling_stats = LinguisticVectorizer()
# all_features = FeatureUnion([('ling', ling_stats), ('tfidf', tfidf_ngrams)])
# clf = MultinomialNB(alpha=5)

# pipeline = Pipeline([('all', all_features), ('clf', clf)])

# pipeline.fit(X_train, y_train)

In [209]:
tfidf_ngrams = TfidfVectorizer(ngram_range=(1,3))
ling_stats = LinguisticVectorizer()
# all_features = FeatureUnion([('ling', ling_stats), ('tfidf', tfidf_ngrams)])
clf = MultinomialNB(alpha=5)

pipeline = Pipeline([('tfidf', tfidf_ngrams), ('clf', clf)])

pipeline.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 3), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('clf',
                 MultinomialNB(alpha=5, class_prior=None, fit_prior=True))],
         verbose=False)

In [210]:
# pd.DataFrame(pipeline.predict(X_test)).value_counts()

In [211]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
y_pred_self = pipeline.predict(X_test)

In [217]:
print('F1 Score: ', f1_score(y_test, y_pred_self))

F1 Score:  0.7678336404316926


In [218]:
sum(y_pred_self == y_test)/len(y_test)

0.7795

In [219]:
tn, fp, fn, tp = confusion_matrix(y_test, y_pred_self).ravel()
(tp, fp, tn, fn)

(2917, 663, 3319, 1101)

In [220]:
confusion_matrix(y_test, y_pred_self)

array([[3319,  663],
       [1101, 2917]])

In [ ]:
tfidf_ngrams = TfidfVectorizer(ngram_range=(1, 3))
# ling_stats = LinguisticVectorizer()
all_features = FeatureUnion([('ling', ling_stats), ('tfidf', tfidf_ngrams)])

clf = LogisticRegression(penalty='l1',
                         solver='saga',
                         multi_class='multinomial',
                         tol=1e-5,
                         n_jobs = -1)

pipeline = Pipeline([('tfidf', tfidf_ngrams), ('clf', clf)])

pipeline.fit(X_train, y_train)
y_pred_lr = pipeline.predict(X_test)
print('F1 Score: ', f1_score(y_test, y_pred_lr, pos_label=4))
sum(y_pred_lr == y_test)/len(y_test)

In [ ]:
tfidf_ngrams = TfidfVectorizer(ngram_range=(1, 3))
# ling_stats = LinguisticVectorizer()
all_features = FeatureUnion([('ling', ling_stats), ('tfidf', tfidf_ngrams)])

clf = LogisticRegression(penalty='elasticnet',
                         solver='saga',
                         multi_class='multinomial',
                         tol=1e-5,
                         n_jobs = -1)

pipeline = Pipeline([('tfidf', tfidf_ngrams), ('clf', clf)])

pipeline.fit(X_train, y_train)
y_pred_lr = pipeline.predict(X_test)
print('F1 Score: ', f1_score(y_test, y_pred_lr, pos_label=4))
sum(y_pred_lr == y_test)/len(y_test)

In [ ]:
pd.DataFrame(y_test)

In [ ]:
res = pd.DataFrame({'Prediction': y_pred_self, 'True':y_test})

In [ ]:
res

In [ ]:
pd.set_option('max_colwidth', 400)

In [ ]:
df.iloc[res[res['Prediction'] != res['True']].index][['Polarity', 'Tweet', 'Tweet_final_sent']]

In [ ]:
df.iloc[res[res['Prediction'] == res['True']].index][['Polarity', 'Tweet', 'Tweet_final_sent']]

In [ ]:
df.iloc[16775]

In [294]:
# set(stopwords.words('english')).difference(['not', 'very'])

In [ ]:
tfidf_ngrams = TfidfVectorizer(min_df=5, ngram_range=(1, 3))
# ling_stats = LinguisticVectorizer()
all_features = FeatureUnion([('tfidf', tfidf_ngrams)])
clf = MultinomialNB(alpha=5)

pipeline = Pipeline([('all', all_features), ('clf', clf)])

pipeline.fit(X_train, y_train)
y_pred_nb = pipeline.predict(X_test)
print('F1 Score: ', f1_score(y_test, y_pred_nb, pos_label=4))
print(sum(y_pred_nb == y_test)/len(y_test))

In [ ]:
len(X_test)

In [ ]:
tfidf_ngrams = TfidfVectorizer(min_df=20, ngram_range=(1, 1))
ling_stats = LinguisticVectorizer()
all_features = FeatureUnion([('ling', ling_stats), ('tfidf', tfidf_ngrams)])

clf = LogisticRegression(penalty='l2',
                         solver='lbfgs',
                         multi_class='multinomial',
                         tol=1e-5,
                         n_jobs = -1)

pipeline = Pipeline([('all', all_features), ('clf', clf)])

pipeline.fit(X_train, y_train)
y_pred_lr = pipeline.predict(X_test)
print('F1 Score: ', f1_score(y_test, y_pred_lr, pos_label=4))
sum(y_pred_lr == y_test)/len(y_test)

In [ ]:
tfidf_ngrams.get_feature_names()

In [ ]:
import pickle

In [ ]:
pickle.dump(pipeline, open('/content/drive/MyDrive/COL772_A2/model_25.txt', 'wb'))

In [ ]:
pos_st = df[df['Polarity'] == 4]['Tweet_normalised'].apply(str.split).sum()
neg_st = df[df['Polarity'] == 0]['Tweet_normalised'].apply(str.split).sum()

In [ ]:
pos_uni_freq = FreqDist(ngrams(pos_st, 1))
neg_uni_freq = FreqDist(ngrams(neg_st, 1))
pos_bi_freq = FreqDist(ngrams(pos_st, 2))
neg_bi_freq = FreqDist(ngrams(neg_st, 2))
pos_tri_freq = FreqDist(ngrams(pos_st, 3))
neg_tri_freq = FreqDist(ngrams(neg_st, 3))

In [ ]:
pos_uni_top = pos_uni_freq.most_common(1000)
neg_uni_top = neg_uni_freq.most_common(1000)
pos_bi_top = pos_bi_freq.most_common(1000)
neg_bi_top = neg_bi_freq.most_common(1000)
pos_tri_top = pos_tri_freq.most_common(1000)
neg_tri_top = neg_tri_freq.most_common(1000)

In [ ]:
pos_uni_top

[(('get',), 3923),
 (('go',), 3437),
 (('laugh',), 3364),
 (('good',), 3222),
 (('love',), 2983),
 (('day',), 2701),
 (('like',), 2011),
 (('thanks',), 1767),
 (('time',), 1686),
 (('well',), 1591),
 (('u',), 1581),
 (('see',), 1562),
 (('today',), 1525),
 (('know',), 1443),
 (('work',), 1413),
 (('make',), 1411),
 (('one',), 1397),
 (('new',), 1373),
 (('think',), 1369),
 (('great',), 1326),
 (('night',), 1209),
 (('watch',), 1192),
 (('back',), 1185),
 (('look',), 1126),
 (('oh',), 1094),
 (('would',), 1056),
 (('twitter',), 1052),
 (('come',), 1041),
 (('morning',), 1016),
 (('happy',), 981),
 (('hope',), 963),
 (('really',), 915),
 (('fun',), 905),
 (('wait',), 898),
 (('much',), 886),
 (('want',), 876),
 (('say',), 871),
 (('need',), 854),
 (('nice',), 818),
 (('home',), 806),
 (('thank',), 805),
 (('take',), 761),
 (('hey',), 757),
 (('tomorrow',), 750),
 (('still',), 716),
 (('yeah',), 703),
 (('tweet',), 698),
 (('follow',), 698),
 (('yes',), 686),
 (('awesome',), 686),
 (('thi

In [ ]:
def get_top_words(sent_list):
    word_list = []
    for i in range(len(sent_list)):
        word_list.append(sent_list[i][0][0])
    return word_list

In [ ]:
pos_uni_top_words = get_top_words(pos_uni_top)
neg_uni_top_words = get_top_words(neg_uni_top)
pos_bi_top_words = get_top_words(pos_bi_top)
neg_bi_top_words = get_top_words(neg_bi_top)
pos_tri_top_words = get_top_words(pos_tri_top)
neg_tri_top_words = get_top_words(neg_tri_top)

In [ ]:
print(len(pos_uni_top_words))
print(len(neg_uni_top_words))


1000
1000


In [ ]:
uni_top_common = set(set(pos_uni_top_words) & set(neg_uni_top_words))
pos_best_words = list(set(pos_uni_top_words) - uni_top_common)
neg_best_words = list(set(neg_uni_top_words) - uni_top_common)

In [ ]:
uni_top_common_list = list(uni_top_common)

In [ ]:
pos_best_words

['aha',
 'usually',
 'interesting',
 'hilarious',
 'nah',
 'proud',
 'choose',
 'lovin',
 'thanx',
 'promise',
 'draw',
 'excited',
 'thankyou',
 'bath',
 'appreciate',
 'gorgeous',
 'wo',
 'ahead',
 'sexy',
 'wop',
 'j',
 'ai',
 'positive',
 'official',
 'review',
 'heh',
 'quote',
 'surprise',
 'germany',
 'hun',
 'pizza',
 'consider',
 'john',
 'sex',
 'war',
 'perfect',
 'profile',
 'topic',
 'matt',
 'choice',
 'demi',
 'fav',
 'design',
 'jk',
 'rd',
 'shal',
 'anyways',
 'indeed',
 'dany',
 'often',
 'congratulation',
 'paint',
 'usual',
 'etc',
 'hanah',
 'experience',
 'adorable',
 'shin',
 'anniversary',
 'aka',
 'brilliant',
 'sweetie',
 'bake',
 'unles',
 'prepare',
 'direct',
 'count',
 'folk',
 'sky',
 'studio',
 'view',
 'quiet',
 'gift',
 'tweps',
 'everybody',
 'space',
 'color',
 'fresh',
 'fabulous',
 'lake',
 'joe',
 'include',
 'bright',
 'peace',
 'info',
 'detail',
 'chilin',
 'treat',
 'michael',
 'doin',
 'wave',
 'american',
 'tip',
 'drinking',
 'secret',
 'k

In [ ]:
neg_best_words

['delay',
 'completely',
 'cough',
 'disappointed',
 'fell',
 'cancel',
 'thunder',
 'dnt',
 'kate',
 'ew',
 'piss',
 'blow',
 'terrible',
 'uh',
 'shift',
 'lonely',
 'throat',
 'nightmare',
 'fever',
 'however',
 'blood',
 'confuse',
 'stuck',
 'starbucks',
 'to',
 'broken',
 'traffic',
 'trouble',
 'sat',
 'somewhere',
 'unfortunately',
 'screw',
 'ring',
 'ac',
 'shitty',
 'bank',
 'rid',
 'board',
 'entire',
 'couldnt',
 'shes',
 'painful',
 'wasnt',
 'wah',
 'floor',
 'rip',
 'crappy',
 'doctor',
 'mouth',
 'freeze',
 'key',
 'dread',
 'blackberry',
 'block',
 'laundry',
 'anywhere',
 'co',
 'exhaust',
 'burnt',
 'vet',
 'bummer',
 'blah',
 'no',
 'bo',
 'atm',
 'there',
 'san',
 'shut',
 'slept',
 'arm',
 'tummy',
 'accident',
 'left',
 'wat',
 'france',
 'knee',
 'shame',
 'ear',
 'bored',
 'guted',
 'duno',
 'science',
 'sadly',
 'ouch',
 'leg',
 'report',
 'fml',
 'aint',
 'headache',
 'assignment',
 'darn',
 'revision',
 'depress',
 'weight',
 'schedule',
 'doesnt',
 'gah',


In [ ]:
len(pos_best_words)

205

In [ ]:
# tfidf_ngrams = TfidfVectorizer(min_df=5, ngram_range=(1, 3))
# # ling_stats = LinguisticVectorizer()
# all_features = FeatureUnion([('tfidf', tfidf_ngrams), ('pos', pos_best_words), ('neg', neg_best_words)])
# clf = MultinomialNB(alpha=1)

# pipeline = Pipeline([('all', all_features), ('clf', clf)])

# pipeline.fit(X_train, y_train)
# y_pred_nb = pipeline.predict(X_test)
# print('F1 Score: ', f1_score(y_test, y_pred_nb, pos_label=4))
# print(sum(y_pred_nb == y_test)/len(y_test))

In [ ]:
df['Tweet'].iloc[50]

In [ ]:
df['Tweet_sent'].iloc[50]

In [ ]:
data.iloc[786897,:]

In [ ]:
df.iloc[786897]

In [ ]:
def dummy(tweet):
    return tweet

In [ ]:
cv = CountVectorizer(  
                      tokenizer=dummy,
                      preprocessor=dummy,
                      ngram_range=(1,1)
                    )

In [ ]:
print("Hi")

In [ ]:
X = df['Polarity', 'Tweet_final_sent']
X_train, X_test, y_train, y_test = train_test_split(X['Tweet_final_sent'], X['Polarity'], test_size=0.25, random_state=2)

In [ ]:
# X = cv.fit_transform(df['Tweet_lemma']).toarray()

In [ ]:
X

In [ ]:
X.shape

In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(X, df['Polarity'], test_size=0.25, random_state=2)

In [ ]:
# X_train = X[:80000,:]
# X_test = X[80000:,:]
# y_train = df['Polarity'][:80000]
# y_test = df['Polarity'][80000:]

In [ ]:
X_train.shape

In [ ]:
X_train

In [ ]:
def model_run(model, X_train, y_train):
    model.fit(X_train, y_train)

In [ ]:
def model_predict(model, X_test, y_test):
    print('Accuracy is: ', model.score(X_test, y_test)*100)
    y_pred = model.predict(X_test)
    print(classification_report(y_test, y_pred))

In [ ]:
model = MultinomialNB()
model_run(model, X_train, y_train)
model_predict(model, X_test, y_test)

In [ ]:
# model = LogisticRegression()
# model_run(model, X_train, y_train)
# model_predict(model, X_test, y_test)

In [ ]:
model_predict(model, X_test, y_test)

In [ ]:
model = LinearSVC()
model_run(model, X_train, y_train)
model_predict(model, X_test, y_test)

In [ ]:
df.head()

In [ ]:
tfidf_counts = TfidfVectorizer(tokenizer= word_tokenize, # type of tokenization
                               ngram_range=(1,1)) # number of n-grams
tfidf_data = tfidf_counts.fit_transform(df['Tweet_sent'])

In [ ]:
# tfidf_counts = TfidfVectorizer()
# tfidf_data = tfidf_counts.fit_transform(a)

In [ ]:
tfidf_data.shape

In [ ]:
X_train_tfidf, X_test_tfidf, y_train_tfidf, y_test_tfidf = train_test_split(tfidf_data, df['Polarity'], test_size=0.25, random_state=2)

In [ ]:
print(X_train_tfidf.shape)
print(X_test_tfidf.shape)
print(y_train_tfidf.shape)
print(y_test_tfidf.shape)

In [ ]:
model = MultinomialNB()
model_run(model, X_train_tfidf, y_train_tfidf)
model_predict(model, X_test_tfidf, y_test_tfidf)

In [ ]:
model = LinearSVC()
model_run(model, X_train_tfidf, y_train_tfidf)
model_predict(model, X_test_tfidf, y_test_tfidf)

In [ ]:
# model = LogisticRegression()
# model_run(model, X_train_tfidf, y_train_tfidf)
# model_predict(model, X_test_tfidf, y_test_tfidf)

In [ ]:
def remove_extra_words(tweet):
    tweet = [word for word in tweet if word in uni_top_common_list]
    if len(tweet) == 0:
        tweet = ['None']
    return tweet

In [ ]:
df['Tweet_remove_extra'] = df['Tweet_lemma'].apply(remove_extra_words)
df.head()

,Polarity,Tweet,Tweet_regex,Tweet_clean,Tweet_stopword,Tweet_clitics,Tweet_shortforms,Tweet_pos,Tweet_lemma,Tweet_sent,Tweet_remove_extra
514293,0,i miss nikki nu nu already shes always there ...,miss nikki nu nu already shes always there whe...,"[miss, nikki, nu, nu, already, shes, always, t...","[miss, nikki, nu, nu, already, shes, always, n...","[miss, nikki, nu, nu, already, shes, always, n...","[miss, nikki, nu, nu, already, shes, always, n...","[(miss, JJ), (nikki, NN), (nu, JJ), (nu, JJ), ...","[miss, nikki, nu, nu, already, shes, always, n...",miss nikki nu nu already shes always need than...,"[miss, already, shes, always, need, thank, xxx]"
142282,0,So I had a dream last night. I remember a sig...,So had dream last night remember sign which cl...,"[So, had, dream, last, night, remember, sign, ...","[So, dream, last, night, remember, sign, clear...","[So, dream, last, night, remember, sign, clear...","[So, dream, last, night, remember, sign, clear...","[(So, RB), (dream, NN), (last, JJ), (night, NN...","[So, dream, last, night, remember, sign, clear...",So dream last night remember sign clearly tell...,"[So, dream, last, night, remember, sign, tell,..."
403727,0,@girlyghost ohh poor sickly you (((hugs)) ho...,ohh poor sickly you hugs hope you feel little ...,"[ohh, poor, sickly, you, hugs, hope, you, feel...","[ohh, poor, sickly, hugs, hope, feel, little, ...","[ohh, poor, sickly, hugs, hope, feel, little, ...","[ohh, poor, sickly, hugs, hope, feel, little, ...","[(ohh, JJ), (poor, JJ), (sickly, JJ), (hugs, N...","[ohh, poor, sickly, hug, hope, feel, little, g...",ohh poor sickly hug hope feel little good soon,"[hug, hope, feel, little, good, soon]"
649503,0,it is raining again,it is raining again,"[it, is, raining, again]",[raining],[raining],[raining],"[(raining, VBG)]",[rain],rain,[rain]
610789,0,@MissKeriBaby wish I was in LA right now,wish was in LA right now,"[wish, was, in, LA, right, now]","[wish, LA, right]","[wish, LA, right]","[wish, LA, right]","[(wish, JJ), (LA, NNP), (right, NN)]","[wish, LA, right]",wish LA right,"[wish, LA, right]"


In [ ]:
df = make_sentences(df, 'Tweet_remove_extra', 'Tweet_final_sent')
df.head()

,Polarity,Tweet,Tweet_regex,Tweet_clean,Tweet_stopword,Tweet_clitics,Tweet_shortforms,Tweet_pos,Tweet_lemma,Tweet_sent,Tweet_remove_extra,Tweet_final_sent
514293,0,i miss nikki nu nu already shes always there ...,miss nikki nu nu already shes always there whe...,"[miss, nikki, nu, nu, already, shes, always, t...","[miss, nikki, nu, nu, already, shes, always, n...","[miss, nikki, nu, nu, already, shes, always, n...","[miss, nikki, nu, nu, already, shes, always, n...","[(miss, JJ), (nikki, NN), (nu, JJ), (nu, JJ), ...","[miss, nikki, nu, nu, already, shes, always, n...",miss nikki nu nu already shes always need than...,"[miss, already, shes, always, need, thank, xxx]",miss already shes always need thank xxx
142282,0,So I had a dream last night. I remember a sig...,So had dream last night remember sign which cl...,"[So, had, dream, last, night, remember, sign, ...","[So, dream, last, night, remember, sign, clear...","[So, dream, last, night, remember, sign, clear...","[So, dream, last, night, remember, sign, clear...","[(So, RB), (dream, NN), (last, JJ), (night, NN...","[So, dream, last, night, remember, sign, clear...",So dream last night remember sign clearly tell...,"[So, dream, last, night, remember, sign, tell,...",So dream last night remember sign tell get job...
403727,0,@girlyghost ohh poor sickly you (((hugs)) ho...,ohh poor sickly you hugs hope you feel little ...,"[ohh, poor, sickly, you, hugs, hope, you, feel...","[ohh, poor, sickly, hugs, hope, feel, little, ...","[ohh, poor, sickly, hugs, hope, feel, little, ...","[ohh, poor, sickly, hugs, hope, feel, little, ...","[(ohh, JJ), (poor, JJ), (sickly, JJ), (hugs, N...","[ohh, poor, sickly, hug, hope, feel, little, g...",ohh poor sickly hug hope feel little good soon,"[hug, hope, feel, little, good, soon]",hug hope feel little good soon
649503,0,it is raining again,it is raining again,"[it, is, raining, again]",[raining],[raining],[raining],"[(raining, VBG)]",[rain],rain,[rain],rain
610789,0,@MissKeriBaby wish I was in LA right now,wish was in LA right now,"[wish, was, in, LA, right, now]","[wish, LA, right]","[wish, LA, right]","[wish, LA, right]","[(wish, JJ), (LA, NNP), (right, NN)]","[wish, LA, right]",wish LA right,"[wish, LA, right]",wish LA right


In [ ]:
tfidf_counts_clean = TfidfVectorizer(tokenizer= word_tokenize, # type of tokenization
                               ngram_range=(1,2)) # number of n-grams
tfidf_data_clean = tfidf_counts_clean.fit_transform(df['Tweet_final_sent'])

In [ ]:
X_train_tfidf, X_test_tfidf, y_train_tfidf, y_test_tfidf = train_test_split(tfidf_data_clean, df['Polarity'], test_size=0.25, random_state=2)

In [ ]:
print(X_train_tfidf.shape)
print(X_test_tfidf.shape)
print(y_train_tfidf.shape)
print(y_test_tfidf.shape)

(30000, 69746)
(10000, 69746)
(30000,)
(10000,)


In [ ]:
model = MultinomialNB()
model_run(model, X_train_tfidf, y_train_tfidf)
model_predict(model, X_test_tfidf, y_test_tfidf)

Accuracy is:  70.34
              precision    recall  f1-score   support

           0       0.71      0.70      0.70      5048
           4       0.70      0.71      0.70      4952

    accuracy                           0.70     10000
   macro avg       0.70      0.70      0.70     10000
weighted avg       0.70      0.70      0.70     10000



In [ ]:
model = LinearSVC()
model_run(model, X_train_tfidf, y_train_tfidf)
model_predict(model, X_test_tfidf, y_test_tfidf)

Accuracy is:  69.61
              precision    recall  f1-score   support

           0       0.71      0.66      0.69      5048
           4       0.68      0.73      0.70      4952

    accuracy                           0.70     10000
   macro avg       0.70      0.70      0.70     10000
weighted avg       0.70      0.70      0.70     10000



In [ ]:
model = LogisticRegression()
model_run(model, X_train_tfidf, y_train_tfidf)
model_predict(model, X_test_tfidf, y_test_tfidf)

Accuracy is:  71.04
              precision    recall  f1-score   support

           0       0.73      0.69      0.71      5048
           4       0.70      0.74      0.72      4952

    accuracy                           0.71     10000
   macro avg       0.71      0.71      0.71     10000
weighted avg       0.71      0.71      0.71     10000



In [ ]:
from sklearn.pipeline import Pipeline

In [ ]:
text_clf = Pipeline([
    ('tfidf',TfidfVectorizer(preprocessor=None,
                             tokenizer=word_tokenize,
                             analyzer='word',
                             stop_words=None,
                             strip_accents=None,
                             lowercase=True,
                             ngram_range=(1,3),
                             min_df=0.0001,
                             max_df=0.9,
                             binary=False,
                             norm='l2',
                             use_idf=1,
                             smooth_idf=1,
                             sublinear_tf=1)),
    ('clf', LogisticRegression(penalty='l2',
                               solver='saga',
                               multi_class='multinomial',
                              tol=1e-5,
                              n_jobs = -1)),
])

In [ ]:

text_clf.fit(X_train,y_train)